# Completing actor data

In [1]:
from src.data.transform_data import* 
from src.models.movie_success_model import* 
from src.models.actor_success_model import*
from webscraping.webscraper import*

## 1) Load character data from csv

In [2]:
raw_df = raw_data()
df = clean_data(raw_df)
movie_success_index(df)
actor_df = actor_data(df)
actor_data = actor_success_index(actor_df)
actor_data

,Actor date of birth,Actor gender,Actor height,Actor ethnicity,Actor age at movie release,Actor Score Index
Actor name,,,,,,
denzel_washington,1954-12-28,M,1.84,/m/0x67,26.0,10.000000
matt_damon,1970-10-08,M,1.78,/m/0cnvdq1,18.0,9.912690
tom_hanks,1956-07-09,M,1.83,/m/02p4q5p,27.0,9.741555
eddie_murphy,1961-04-03,M,1.75,/m/0x67,21.0,9.415060
tom_cruise,1962-07-03,M,1.70,/m/02ctzb,18.0,9.362255
...,...,...,...,...,...,...
j._kenneth_campbell,1947-07-22,M,NaN,None,43.0,0.714754
evelyn_keyes,1916-11-20,F,1.63,None,72.0,0.700949
james_dixon,None,M,NaN,None,NaN,0.700949


## 3) Activate scraper

In [3]:
test_sample = actor_data[:50]
spider = ActorScraper(test_sample)
test_sample = spider.run_scraping()
test_sample

/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'Gender'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:318: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.actor_data.loc[:, 'University'] = None
/Users/aiden/Desktop/EPFL/Master/M - ADA/ada-2024-project-sigma-squad/webscraping/webscraper.py:319: SettingWithCopyWarning: 
A value 

body found


100%|██████████| 50/50 [02:00<00:00,  2.40s/it]

Scraping completed.


Actor date of birth Actor gender  Actor height  \
Actor name                                                             
Denzel Washington              1954-12-28            M         1.840   
Matt Damon                     1970-10-08            M         1.780   
Tom Hanks                      1956-07-09            M         1.830   
Eddie Murphy                   1961-04-03            M         1.750   
Tom Cruise                     1962-07-03            M         1.700   
Frank Welker                   1946-03-12            M         1.730   
Harrison Ford                  1942-07-13            M         1.850   
James Woods                    1947-04-18            M         1.803   
Michelle Pfeiffer              1958-04-29            F         1.710   
Trey Parker                    1969-10-19            M         1.850   
Samuel L. Jackson              1948-12-21            M         1.890   
Richard Jenkins                1947-05-04            M         1.850   
Danny Devito                   1944-11-17            M         1.520   
Kirsten Dunst                  1982-04-30            F         1.700   
Chris Rock                     1965-02-07            M         1.800   
Robert De Niro                 1943-08-17            M         1.770   
George Clooney                 1961-05-06            M         1.800   
Jim Carrey                     1962-01-17            M         1.867   
Kevin Costner                  1955-01-18            M         1.850   
Stanley Tucci                  1960-11-11            M         1.721   
Clint Eastwood                 1930-05-31            M         1.880   
Jonah Hill                     1983-12-20            M         1.700   
Nicole Kidman                  1967-06-20            F         1.791   
Arnold Schwarzenegger          1947-07-30            M         1.880   
Mark Wahlberg                  1971-06-05            M         1.700   
Hank Azaria                    1964-04-25            M         1.820   
Sylvester Stallone             1946-07-06            M         1.770   
Steve Carell                   1962-08-16            M         1.750   
Jack Nicholson                 1937-04-22            M         1.770   
Val Kilmer                     1959-12-31            M         1.830   
Harry Shearer                  1943-12-23            M         1.670   
Hugh Jackman                   1968-10-12            M         1.890   
Wesley Snipes                  1962-07-31            M         1.753   
Halle Berry                    1966-08-14            F         1.680   
Jean-Claude Van Damme          1960-10-18            M         1.780   
Edward Norton                  1969-08-18            M         1.830   
Ewan McGregor                  1971-03-31            M         1.790   
John Ratzenberger              1947-04-06            M         1.750   
Jeff Bridges                   1949-12-04            M         1.850   
Dennis Quaid                   1954-04-09            M         1.830   
Helena Bonham Carter           1966-05-26            F         1.600   
Scarlett Johansson             1984-11-22            F         1.630   
Tommy Lee Jones                1946-09-15            M         1.830   
Martin Lawrence                1965-04-16            M         1.710   
Catherine O%27Hara             1954-03-04            F         1.630   
Bruce McGill                   1950-07-11            M         1.760   
Jackie Chan                    1954-04-07            M         1.740   
James Cromwell                 1940-01-27            M         2.010   
Hugo Weaving                   1960-04-04            M         1.880   
Steve Martin                   1945-08-14            M         1.830   

                      Actor ethnicity  Actor age at movie release  \
Actor name                                                          
Denzel Washington             /m/0x67                        26.0   
Matt Damon                 /m/0cnvdq1                        18.0   
T